In [1]:
import random
import wandb
import numpy as np
from dotmap import DotMap

import torch
import torch.nn as nn
import torch.optim as optim

# Tensor decomposition
from torch.utils.data import DataLoader

import sys
from costco import *
from read import *
from utils import *

import matplotlib.pyplot as plt

In [2]:
device = 'cpu'

In [3]:
work_dir = "/Users/shaanpakala/Desktop/NSF_REU_2024/Research/"
data_folder = f"{work_dir}classification_datasets/"

t1 = torch.load(f"{work_dir}meta_datasets/FCNN_glass_717.pt")
t2 = torch.load(f"{work_dir}meta_datasets/FCNN_car_evaluation_717.pt")

original_tensor = torch.stack((t1, t2))
t = original_tensor.clone()

del t1, t2

rand_index = lambda x: tuple([int(random.random()*i) for i in x])

def get_indices(shape, num_indices):
    
    index_bank = dict()
    
    while (len(index_bank) < num_indices):
        index_bank[rand_index(shape)] = True
        
    return list(index_bank)

total_cells = 1
for s in t.shape: total_cells*=s

print(f"{total_cells} total cells in tensor.")

portion_of_entries = 0.05
indices = get_indices(t.shape, int(total_cells*(1-portion_of_entries)))
for index in indices: t[index] = 0
    
print(f"{int(total_cells*(portion_of_entries))} non-zero cells.")

5760 total cells in tensor.
288 non-zero cells.


In [4]:
sparse_tensor = t.to_sparse()

In [5]:
indices = sparse_tensor.indices().t().to(device) # NNZ x mode
values = sparse_tensor.values().to(device) # NNZ
values = values.to(torch.double)

In [6]:
tensor_size = sparse_tensor.size()

In [7]:
# indices.shape

# 1. Read data

In [8]:
from torch.utils.data import Dataset
class COODataset(Dataset):
    def __init__(self, idxs, vals):
        self.idxs = idxs
        self.vals = vals

    def __len__(self):
        return self.vals.shape[0]

    def __getitem__(self, idx):
        return self.idxs[idx], self.vals[idx]

In [9]:
batch_size = 1024
dataset = COODataset(indices, values)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 2. Build a model

In [10]:
class CoSTCo(nn.Module):
    def __init__(self, cfg):
        super(CoSTCo, self).__init__()
        nc = cfg.nc
        self.rank = cfg.rank
        self.sizes = cfg.sizes
        self.embeds = nn.ModuleList([nn.Embedding(self.sizes[i], self.rank)
                                     for i in range(len(self.sizes))])
        self.conv1 = nn.Conv2d(1, nc, kernel_size=(1, len(self.sizes)), padding=0)
        self.conv2 = nn.Conv2d(nc, nc, kernel_size=(self.rank, 1), padding=0)
        self.fc1 = nn.Linear(nc, nc)
        self.fc2 = nn.Linear(nc, 1)
        self.relu = nn.ReLU()
        self._initialize()
        
    def _initialize(self):
        
        for i in range(len(self.embeds)):
            nn.init.kaiming_uniform_(self.embeds[i].weight.data)
        
    def forward(self, inputs):
        '''
        inputs: indices of nnz
        '''
        
        embeds = [self.embeds[m](inputs[:, m]).reshape(-1, self.rank, 1)
                  for m in range(len(self.sizes))]
        x = torch.cat(embeds, dim=2)
        x = x.reshape(-1, 1, self.rank, len(self.sizes))# NNZ_batch x 1 x rank x nmode 
        
        # CNN on stacked embeds
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        
        # Aggregate with mlps
        x = x.view(-1, x.size(1))
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        
        return x.reshape(-1)

In [11]:
# cfg.loss = 'bceloss'
# cfg.rank = 8
# cfg.nc = 16
# cfg.lr = 1e-3
# cfg.wd = 1e-5
# cfg.epochs = 100
# batch_size = 1024

In [12]:
cfg = DotMap()
cfg.nc = 8
cfg.rank = 3
cfg.sizes = tensor_size
cfg.lr = 0.0001
cfg.wd = 0.0001
cfg.epochs = 1000

In [13]:
# create the model
model = CoSTCo(cfg).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.wd)

In [14]:
flag = 0
err_lst = []
old_rmse = 1e+6
# train the model
for epoch in range(cfg.epochs):

    model.train()
#     epoch_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        inputs, targets = batch[0], batch[1]
        outputs = model(inputs)
        loss = loss_fn(outputs.to(torch.double), targets)
        loss.backward()
        optimizer.step()
        # epoch_loss += loss.item()
    
    model.eval()
    if (epoch+1) % 100 == 0:
        with torch.no_grad():
            train_rec = model(indices)
            train_rmse = rmse(train_rec, values)

            print(f"Epoch {epoch+1} Train RMSE: {train_rmse:.4f}\t")

            if (old_rmse <= train_rmse):
                flag +=1
            if flag == 10:
                break
            old_rmse = train_rmse

Epoch 100 Train RMSE: 0.4626	
Epoch 200 Train RMSE: 0.4154	
Epoch 300 Train RMSE: 0.3720	
Epoch 400 Train RMSE: 0.3308	
Epoch 500 Train RMSE: 0.2924	
Epoch 600 Train RMSE: 0.2608	
Epoch 700 Train RMSE: 0.2387	
Epoch 800 Train RMSE: 0.2260	
Epoch 900 Train RMSE: 0.2196	
Epoch 1000 Train RMSE: 0.2160	


In [15]:
def get_unique(all_indices, train_indices):

    unique_dict = dict()

    for i in [tuple([int(y) for y in x]) for x in all_indices]:
        unique_dict[i] = True

    for j in [tuple([int(y) for y in x]) for x in train_indices]:
        del unique_dict[j]

    unique_indices = torch.tensor([list(x) for x in list(unique_dict)])

    del unique_dict
    
    return unique_indices

In [29]:
def get_unique_MAE(tensor_completion_model, full_t, sparse_t):
    
    unique_indices = get_unique(full_t.to_sparse().indices().t(), sparse_t.indices().t())
    unique_recon = model(unique_indices)

    unique_recon_MAE = 0
    for i in range(len(unique_indices)):
        unique_index = tuple(unique_indices[i])
        unique_value = unique_recon[i]

        unique_recon_MAE += float(abs(unique_value - original_tensor[unique_index]))

    unique_recon_MAE /= len(unique_indices)
    
    return unique_recon_MAE

In [27]:
unique_indices = get_unique(original_tensor.to_sparse().indices().t(), sparse_tensor.indices().t())
unique_recon = model(unique_indices)

unique_recon_MAE = 0
for i in range(len(unique_indices)):
    unique_index = tuple(unique_indices[i])
    unique_value = unique_recon[i]

    unique_recon_MAE += float(abs(unique_value - original_tensor[unique_index]))

unique_recon_MAE /= len(unique_indices)

In [30]:
get_unique_MAE(tensor_completion_model = model,
               full_t = original_tensor,
               sparse_t = sparse_tensor)

0.19085306875266042

In [19]:
with torch.no_grad(): 
    test_rec =  model(test_indices)
    test_rmse = rmse(test_values)

    print(f"TestRMSE:{test_rmse:.4f}")
    #     wandb.log({"test_rmse":dct.test_rmse, "acc":dct.acc, "recall": dct.recall,
    #                "prec": dct.prec, "f1":dct.f1, "auc":dct.auc})

NameError: name 'test_indices' is not defined

In [ ]:
exp_name = f"{name}_rank{cfg.rank} lr{cfg.lr} wd{cfg.wd} nc:{cfg.nc}"
exp_title = f"{name} || r:{cfg.rank} lr:{cfg.lr} nc:{cfg.nc}"

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 3), dpi = 200)
plt.plot(np.vstack(err_lst)[:, 0])
plt.plot(np.vstack(err_lst)[:, 1])
plt.title(f"{exp_title} \n TestRMSE:{test_rmse:.4f} Acc:{r['acc']:.4f} AUC : {r['auc']:.4f}")
plt.tight_layout()

# plt.savefig(f'training_results/{exp_name}_costco_training_loss.pdf')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
factors = [model.embeds[i].weight.data.cpu().numpy() for i in range(3)]
plt.figure(figsize=(13, 4), dpi = 300)
for i in range(1, 4):
    plt.subplot(1, 3, i)
    fmat = factors[i-1]
    sns.heatmap(fmat.T @ fmat, cmap='vlag')
    plt.xlabel('Rank')
    plt.ylabel('Rank')
plt.tight_layout()
# plt.savefig(f'training_results/{exp_name}_costco_factors_corr.pdf')

In [ ]:
plt.figure(figsize=(13, 4))
for i in range(1, 4):
    plt.subplot(1, 3, i)
    fmat = factors[i-1]
    sns.heatmap(fmat)
    plt.xlabel('Rank')
    plt.ylabel('Dimensionality')
plt.tight_layout()
# plt.savefig(f'training_results/{exp_name}_costco_factors_corr.pdf')